In [1]:
# Habilitar GPU (ve a Entorno de ejecución > Cambiar tipo de entorno > GPU)
!nvidia-smi  # Verifica que la GPU está activa (deberías ver una tarjeta como T4)

# Crear directorio de trabajo
!mkdir -p TripoSR
%cd TripoSR

Sun May 18 03:59:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Clonar el repositorio de TripoSR
!git clone https://github.com/VAST-AI-Research/TripoSR.git
%cd TripoSR

# Instalar dependencias básicas
!pip install -r requirements.txt

# Instalar PyTorch con soporte CUDA
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

# Instalar versiones específicas de huggingface_hub, accelerate y transformers
!pip install huggingface_hub==0.20.3 accelerate==0.27.2 transformers==4.38.2

# Instalar herramientas adicionales y onnxruntime
!pip install trimesh pythreejs onnxruntime


Cloning into 'TripoSR'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 161 (delta 63), reused 42 (delta 42), pack-reused 66 (from 1)
Receiving objects: 100% (161/161), 36.71 MiB | 18.96 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/TripoSR/TripoSR
  Cloning https://github.com/tatsy/torchmcubes.git to /tmp/pip-req-build-easnqmf4
  Running command git clone --filter=blob:none --quiet https://github.com/tatsy/torchmcubes.git /tmp/pip-req-build-easnqmf4
  Resolved https://github.com/tatsy/torchmcubes.git to commit 3381600ddc3d2e4d74222f8495866be5fafbace4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 5.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tokenizers to determine which version is compatible with other r

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 4.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tokenizers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 117.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 107.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
  Attempting uninstall: transform

In [3]:
# Descargar el modelo preentrenado
!wget https://huggingface.co/stabilityai/TripoSR/resolve/main/model.ckpt -O model.ckpt


--2025-05-18 04:08:28--  https://huggingface.co/stabilityai/TripoSR/resolve/main/model.ckpt
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.17, 18.164.174.23, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/ea/97/ea971396100dd68399edacd9eb8a96a37061014bde74f142916877c3a05305da/429e2c6b22a0923967459de24d67f05962b235f79cde6b032aa7ed2ffcd970ee?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.ckpt%3B+filename%3D%22model.ckpt%22%3B&Expires=1747544909&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NzU0NDkwOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2VhLzk3L2VhOTcxMzk2MTAwZGQ2ODM5OWVkYWNkOWViOGE5NmEzNzA2MTAxNGJkZTc0ZjE0MjkxNjg3N2MzYTA1MzA1ZGEvNDI5ZTJjNmIyMmEwOTIzOTY3NDU5ZGUyNGQ2N2YwNTk2MmIyMzVmNzljZGU2YjAzMmFhN2VkMmZmY2Q5NzBlZT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l

In [4]:
from google.colab import files

# Subir múltiples imágenes
uploaded = files.upload()

# Guardar las imágenes subidas en una lista
image_files = list(uploaded.keys())
print(f"Imágenes subidas: {image_files}")

Saving ChatGPT Image 5 may 2025, 01_22_22 p.m..png to ChatGPT Image 5 may 2025, 01_22_22 p.m..png
Imágenes subidas: ['ChatGPT Image 5 may 2025, 01_22_22 p.m..png']


In [5]:
import os
import shutil

# Crear carpeta para los resultados
output_dir = "output_models"
os.makedirs(output_dir, exist_ok=True)

# Procesar cada imagen con TripoSR
for img_file in image_files:
    output_path = os.path.join(output_dir, f"{os.path.splitext(img_file)[0]}_model.obj")
    print(f"Procesando {img_file}...")
    !python run.py "{img_file}" --output-dir "{output_dir}" --model-save-format obj --device cuda

    # Verificar si se generó el archivo '0' y renombrarlo
    default_output = os.path.join(output_dir, "0")
    if os.path.exists(default_output):
        shutil.move(default_output, output_path)
        print(f"Éxito: Modelo generado y renombrado para {img_file}: {output_path}")
    else:
        print(f"Fallo: No se generó el modelo para {img_file} en {output_path}")
        print("Archivos en output_models después de este intento:", os.listdir(output_dir))


Procesando ChatGPT Image 5 may 2025, 01_22_22 p.m..png...
2025-05-18 04:11:26,862 - INFO - Initializing model ...
config.yaml: 100% 987/987 [00:00<00:00, 6.16MB/s]
model.ckpt: 100% 1.68G/1.68G [00:12<00:00, 130MB/s]
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
config.json: 100% 454/454 [00:00<00:00, 3.14MB/s]
2025-05-18 04:11:52,988 - INFO - Initializing model finished in 26126.37ms.
2025-05-18 04:11:52,988 - INFO - Processing images ...
100%|███████████████████████████████████████| 176M/176M [00:00<00:00, 1.11TB/s]
2025-05-18 04:11:58,666 - INFO - Processing images finished in 5677.61ms.
2025-05-18 04:11:58,666 - INFO - Running image 1/1 ...
2025-05-18 04:11:58,666 - INFO - Running model ...
2025-05-18 04:12:00,466 - INFO - Running model finished in 1799.94ms.
2025-05-18 04:12:00,46

In [6]:
from google.colab import files
import os

# Descargar cada modelo .obj
output_dir = "output_models"
for img_file in image_files:
    model_dir = os.path.join(output_dir, f"{os.path.splitext(img_file)[0]}_model.obj")
    obj_file = os.path.join(model_dir, "mesh.obj")

    print(f"Procesando: {obj_file}")
    if os.path.exists(obj_file):
        print(f"Archivo encontrado: {obj_file}, tamaño: {os.path.getsize(obj_file)} bytes")
        files.download(obj_file)
    else:
        print(f"Archivo no encontrado: {obj_file}")
        print(f"Contenido de {model_dir}:", os.listdir(model_dir) if os.path.exists(model_dir) else "Carpeta no encontrada")

Procesando: output_models/ChatGPT Image 5 may 2025, 01_22_22 p.m._model.obj/mesh.obj
Archivo encontrado: output_models/ChatGPT Image 5 may 2025, 01_22_22 p.m._model.obj/mesh.obj, tamaño: 7114960 bytes


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>